# Direct geometry for 2d robots

This notebook aims at illustrating a kinematic tree and discussing the notions of forward geometry. The Pinocchio library natively provides some of the elements we are going to develop, but we will do without it to get a better understanding of how these notions are concretely applied. 

We will use a basic geometry API to build a robot and display it within the in-browser viewer meshcat.

**Important: to make sure your repository is easily updated from git shall it be needed, create a copy of this notebook before working on it.**

In [1]:
import magic_donotload

NB: as for all the tutorials, a magic command %do_not_load is introduced to hide the solutions to some questions. Change it for %load if you want to see (and execute) the solution.


## Set up
We will need NumPy and MeshCat Viewer for vizualizing the robot. We will also use a helper function later in the tutorial.

In [2]:
# %load tp1/generated/configuration_reduced_import
import time
import numpy as np
from scipy.optimize import fmin_bfgs,fmin_slsqp
from utils.meshcat_viewer_wrapper import MeshcatVisualizer,translation2d,planar
from numpy.linalg import norm,inv,pinv,svd,eig
from pinocchio.rpy import matrixToRpy
import pinocchio as pin

<a id='section_display_objects'></a>
## Displaying objects
Let's first learn how to open a 3D viewer, in which we will build our simulator. We will use the viewer MeshCat which directly displays in a browser. Open it as follows:

In [3]:
viz = MeshcatVisualizer()
viz.viewer.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


You can click on the link above to open the client in a new window, which will be more convenient for seeing your results as you go.

The obtained <viz> object is a client of the viewer, i.e. it will be use to pass display command to the viewer. 
    

**warning**: if you run again the cell above, you will create a new client and the connection to the former will be lost.
   
Take so time to familiarise yourself with the GUI with the mouse, holding the left / right buttons of the mouse and using the scroll wheel.
    
Let's now populate our scene:

In [4]:
cylID  = 'world/cyl' ;  viz.addCylinder(cylID,length=1,radius=.1,color=[0,0,1,1])
ballID = 'world/ball';  viz.addSphere(ballID,.2,[1,0,0,1])
boxID  = 'world/box' ;  viz.addBox(boxID,[.5,.2,.4],[1,1,0,1])

You can take some time to play with the parameters and understand their meaning.

The \*ID strings serve as unique identifier to the object.
You can recreate objects using the same ID, which will simply replace your object by another one. If you want to delete one object, simply run:


In [5]:
viz.delete(ballID)

Placing objects can be done using the applyConfiguration command, and specifying a displacement as list of 7 values. The first three values indicate the 3D position of the **center of the object**. The last 4 values are a normalised quaternion that describes the 3D orientation of the object.

In [6]:
viz.applyConfiguration(cylID,[.1,.2,.3,1,0,0,0])

We will first work in 2D. 
Usually when working in 2D, we refer to the x/y plane, with rotations happening around the z plane.
We will work under these assumptions.

However, since the z plan is pointing upward, for convenience the display will happen in the y/z planes, and rotations around the x axis. This is all handled internally thanks to the *planar* helper function provided for you. If you are curious please do explore the code for planar in the utils folder.

In summary, to place an object given a triplet x,y,theta, we can call:

In [7]:
viz.applyConfiguration(boxID,planar(0.1, 0.2, np.pi / 3))
viz.applyConfiguration(cylID,planar(0.1, 0.2, 5*np.pi / 6))

That's all we need for now regarding the GUI. Now let's move to the actual forward geometry problem

## Forward geometry on a 2D robot
We are going to model the kinematic tree of a 2D manipulator robot. Our objective is to compute the end-effector position given a configuration of the robot.


### Robot model of a 3-Link Planar Manipulator

<table>
    <tr>
        <td><img src="tp1/3-link-angles.png" style="width: 300px;"> </td>
        <td><img src="tp1/3-link-frames.png" style="width: 280px;"> </td>
    </tr>
    <tr>
        <td style="text-align:center">Diagram of the links and joints</td>
        <td style="text-align:center">Diagram of the frames</td>
    </tr>
</table>





Consider the planar manipulator indicated in the above figure. It consists in a set of rigid bodies constrained together by joints (in this case, they are all revolute joints around the z axis).

The respective length of each body 1, 2 and 3 is given by the constants $l_1, l_2$ and $l_3 \in \mathbb{R}$. The current rotation about each of the three joints is given by ${\theta}_1, {\theta}_2$ and ${\theta}_3 \in [-\pi,\pi]$. To each body $i$ a frame is conveniently placed to minimise the calculations. The **frame {O}** is constant (e.g. a point on the ground), and we make it coincide with the origin. The **end-effector** $\mathbf{e} \in \mathbb{R}^3$ is the manipulator attached at the end of the body {C} which is the black gripper.


Let us create the three bodies that compose our robot, but first let us clear the scene:


In [8]:
viz.clean() #be careful, this deletes the whole scene

In [9]:
# %load tp1/generated/configuration_reduced_create
bodynames  = ['body'+str(i) for i in range(4)]
jointnames = ['joint'+str(i) for i in range(4)]

#defining some constants
# feel free to play with these
l1 = l2 = .75
l3 = 0.4
bodywidth = 0.05 
rootwidth = bodywidth * 3
jointwidth = 0.08

#colors
bodycolor  = [.65,.65,.65,1] #grey
jointcolor = [1,0,0,1] #red

#creating bodies
viz.addBox     (bodynames[0],[rootwidth for _ in range(3)],bodycolor)
viz.addCylinder(bodynames[1],l1,bodywidth,bodycolor)
viz.addCylinder(bodynames[2],l2,bodywidth,bodycolor)
viz.addCylinder(bodynames[3],l3,bodywidth,bodycolor)
    
#creating joints
for i in range(4):
    viz.addSphere(jointnames[i],jointwidth,jointcolor)
    
#end-effector marker
viz.addSphere("effector",jointwidth,[0,0,1,1]) #blue


### Homogeneous Transformation Matrix

We will use the homogeneous transformation matrix representation to conveniently group the rotation and translation of a frame into a single matrix form: 
<br/> 
<br/> 
$$
\mathbf{M} =
\begin{bmatrix}
\mathbf{R} & \mathbf{p} \\
\mathbf{0}_{1x3} & 1
\end{bmatrix} 
=
\begin{bmatrix}
r_{11} & r_{12} & r_{13} & x \\
r_{21} & r_{22} & r_{23} & y \\
r_{31} & r_{32} & r_{33} & z \\
0 & 0 & 0 & 1
\end{bmatrix} 
$$

<br/>


<img src="tp1/frames.png" style="width: 600px;">


Given the position of a $^B\mathbf{p}$ in a frame {B}, the position $^A\mathbf{p}$ in the frame {A} is given by the relationship $\newline$

$$
\begin{bmatrix}
^A\mathbf{p} \\
1
\end{bmatrix}  
 = {^A}\mathbf{M}_B
\begin{bmatrix}
^B\mathbf{p} \\
1
\end{bmatrix} 
$$ 

which is equivalent to $\newline$

<center>$^A\mathbf{p} = {^A}\mathbf{R}_B {^B\mathbf{p}} +  \overrightarrow{\mathbf{o}_A\mathbf{o}_B}$ </center> 


In our case, we will only be working with 2D rotations, such that the transformations can be defined using one less dimension:

<br/>
$$
\mathbf{M} =
\begin{bmatrix}
\mathbf{R} & \overrightarrow{\mathbf{o}_A\mathbf{o}_B} \\
\mathbf{0}_{1x2} & 1
\end{bmatrix} 
=
\begin{bmatrix}
r_{11} & r_{12} & x \\
r_{21} & r_{22} & y \\
0 & 0 & 1 
\end{bmatrix} 
$$

<br/>
All the rotations are defined around the z axis, such that the rotation matrices are defined straightforwardly as a function of the rotation angle $\theta$:

$$
\mathbf{R}(\theta)
=
\left(\begin{array}{cc}
cos(\theta) & -sin(\theta) & 0\\
sin(\theta) & cos(\theta) & 0 \\
0 &  0 & 1 \\
\end{array}\right) 
$$


The position ${^C}\mathbf{e}$ is constant. Give its expression (you can load the answer if you need it)

In [10]:
# %do_not_load tp1/generated/ce
ce = np.array([0,l3,0,1.])

Lets write the helper function *R(theta)*, which computes the rotation matrix $\mathbf{R}(\theta)$:

In [11]:
def R_z(theta):
    c = np.cos(theta)
    s = np.sin(theta)
    return np.array([[c,-s,0], [s,c,0], [0,0,1]])
def R_y(theta):
    c = np.cos(theta)
    s = np.sin(theta)
    return np.array([[c,0,s], [0,1,0], [-s,0,c]])
def R_x(theta):
    c = np.cos(theta)
    s = np.sin(theta)
    return np.array([[1,0,0], [0,c,-s], [0,s,c]])

Now provide the code for a function *bMc(theta3)*, which computes the matrix ${^B}\mathbf{M}_C$ as a function ot $\theta_3$

In [12]:
# %do_not_load tp1/generated/bMc
def bMc(theta3):
    M = np.zeros((4,4)) #4*4 matrix
    M[:3,:3] = R_x(theta3) # replace with rotation matrix
    M[:3, 3] = [0, l2, 0]# replace with translation part
    M[3,3] = 1
    return M

Define the remaining transformations in a similar fashion

In [13]:
# %do_not_load tp1/generated/xMy
# %do_not_load tp1/generated/bMc
def aMb(theta2):
    M = np.zeros((4,4)) #4*4 matrix
    M[:3,:3] = R_z(theta2) # replace with rotation matrix
    M[:3, 3] = [0, l1, 0]# replace with translation part
    M[3,3] = 1
    return M

# %do_not_load tp1/generated/bMc
def oMa(theta1):
    M = np.zeros((4,4)) #4*4 matrix
    M[:3,:3] = R_x(theta1) # replace with rotation matrix
    M[:3, 3] = [0, 0, 0]# replace with translation part
    M[3,3] = 1
    return M

Given a configuration vector q of dimension 3, we can now compute the position of the centers of each object using forward geometry and correctly display the robot

In [14]:
# %load tp1/generated/configuration_reduced_display
#computes all the transformation matrices and returns the end effector position in the origin frame
def forwardgeometry(q):
    return [oMa(q[0]),aMb(q[1]),bMc(q[2])]

#computes the end effector position
def effector(q):
    return oMa(q[0]) @ aMb(q[1]) @ bMc(q[2]) @ ce

def sgn(b):
    return np.sign(b)

def rotation2quaternion(R):
    '''Convert a Rotation matrix into it's equivalent unit quaternion'''
    return pin.Quaternion(R).coeffs().tolist().copy()
#     The following are the calculations for this conversion I got from Robotics: Modelling, Planning and Control 
#     phi = 0.5 * np.sqrt(R[0,0] + R[1,1] + R[2,2] + 1)
#     ijk = [-0.5 * sgn(R[2,1] - R[1,2]) * np.sqrt(R[0,0] - R[1,1] - R[2,2] + 1), 
#            0.5 * sgn(R[0,2] - R[2,0]) * np.sqrt(R[1,1] - R[2,2] - R[0,0] + 1), 
#            0.5 * sgn(R[1,0] - R[0,1]) * np.sqrt(R[2,2] - R[0,0] - R[1,1] + 1)
#           ]
#     return ijk + [phi]

def compute_joint_config(jointname, jointPos):
        #display joint i
        viz.applyConfiguration(jointname,
                               [jointPos[0], jointPos[1], jointPos[2]] 
                               + [0,0,0,0])
        
def compute_body_config(bodyname, bodycenter, rotation):
        #display body i
        body_orientation = rotation2quaternion(rotation[:3,:3].copy())
        viz.applyConfiguration(bodyname,
                               [bodycenter[0], bodycenter[1], bodycenter[2]] 
                               + body_orientation)

def display(q):
    '''Compute forward geometry and 
    Display the robot in the viewer. '''
    xMys = forwardgeometry(q.copy())
    currenttransform = np.eye(4)
    currentjointpos = np.zeros(3)
    transformation_array = []
    
    new_joint_pos = oMa(q[0])[:3, -1]
    compute_joint_config(jointnames[0], new_joint_pos)
    new_body_orientation = np.eye(4)# Edit here for box rotation
    new_bodycenter = [0,0,0] #currentjointpos + (new_joint_pos - currentjointpos) /2
    compute_body_config(bodynames[0], new_bodycenter, new_body_orientation )
    
    
    currentjointpos = new_joint_pos
    new_joint_pos = ( oMa(q[0]) @ aMb(q[1]) )[:3, -1]
    compute_joint_config(jointnames[1], new_joint_pos)
    new_bodycenter = currentjointpos + (new_joint_pos - currentjointpos) /2
    new_body_orientation = ( oMa(q[0]) )
    compute_body_config(bodynames[1], new_bodycenter, new_body_orientation )
    

    currentjointpos = new_joint_pos
    new_joint_pos = ( oMa(q[0]) @ aMb(q[1]) @ bMc(q[2]))[:3, -1]
    compute_joint_config(jointnames[2], new_joint_pos)
    new_bodycenter = currentjointpos + (new_joint_pos - currentjointpos) /2
    new_body_orientation = ( oMa(q[0]) @ aMb(q[1]) )
    compute_body_config(bodynames[2], new_bodycenter, new_body_orientation )
    currentjointpos = new_joint_pos

    currentjointpos = new_joint_pos
    new_joint_pos = ( oMa(q[0]) @ aMb(q[1]) @ bMc(q[2]) @ ce)[:3]
    compute_joint_config("effector", new_joint_pos)
    new_bodycenter = currentjointpos + (new_joint_pos - currentjointpos) /2
    new_body_orientation = ( oMa(q[0]) @ aMb(q[1]) @ bMc(q[2]) )
    compute_body_config(bodynames[3], new_bodycenter, new_body_orientation )
    currentjointpos = new_joint_pos        


In [15]:
q = np.random.rand(3) * np.pi *2 - np.pi # angles between pi and -pi
q= np.array([0.35,0.87,0.78])*np.pi
display(q) # Display the robot under configuration q in the viewer

## To go further
Here are a couple of variations you can decide to investigate to make sure you understood how forward geometry works:

+ Add the root of the robot at a different frame (position + orientation) than the origin 
+ In our choice of frame the body is aligned with one axis. Choose arbitrary frames for which the body is not aligned with any axis and work out the forward geometry equations.
+ Add a prismatic joint at the end of body C and update the display function accordingly (ie the length of object used to represent the joint must change as the parameter changes).
+ Go 3D: assign a different axis of rotation to each joint and work out again the equations

We won't give the solutions for these cases but you are welcome to discuss them on Piazza